# Bosonic.py

In [1]:
import numpy as np
from scipy.special import binom 
from scipy.sparse import dok_matrix, linalg
from scipy import linalg as linalg_d
from joblib import Memory
import random
import plotly.graph_objects as go

location = './cachedir'
memory = Memory(location, verbose=0)

In [2]:
class fixed_basis:

    # Convierte a un enterno n a su escritura en base b
    def int_to_tuple(self, n, b, digits = None):
        r = []
        re = n 
        while re != 0:
            r.append(re%b)
            re = re // b
        if digits is not None:
            if len(r)<digits:
                for i in range(0,digits-len(r)):
                    r.append(0)
        r.reverse()
        return r
    
    # Revierte la transformacion anterior
    def tuple_to_int(self, t):
        b = self.d-1
        l = len(t)
        s = [t[k]*b**(l-k-1) for k in range(0,l)]
        return sum(s)

    # Convierte el vector en su representacion
    def vect_to_repr(self, vect):
        for i, k in enumerate(vect):
            if k == 1. or k == 1:
                break
        return self.base[i,:]
            
    def rep_to_vect(self, rep):
        rep = list(rep)
        for i, r in [(j, self.base[j,:]) for j in range(0,self.size)]:
            if list(r) == rep:
                return self.canonicals[:,i]
        else:
            None
    
    def rep_to_index(self, rep):
        rep = list(rep)
        for i, r in [(j, self.base[j,:]) for j in range(0,self.size)]:
            if list(r) == rep:
                return i
        else:
            None
        
    # Crea base de M particulas en D estados (repr y base canonica)
    def create_basis(self, m, d):
        length = int(binom(d+m-1,d-1))
        base = []
        index = 0
        for x in range(0,(d+1)**max(d,m)):
            x = self.int_to_tuple(x,m+1,d)
            if sum(x) == m and len(x) == d:
                base.append(x)
                index += 1
        base = np.array(base)
        # Asignamos a cada uno de ellos un canónico
        x = [1.0 for j in range(0,length)]
        canonicals = np.diag(x)
        return base, canonicals
    
    def __init__(self, m, d):
        self.m = m
        self.d = d
        self.size = int(binom(d+m-1,d-1))
        (self.base, self.canonicals) = self.create_basis(m, d)

# Matrices de aniquilación y creación endomórficas. Estan fuera de la clase para poder ser cacheadas        
@memory.cache    
def bdb(basis, i, j):
    mat = dok_matrix((basis.size, basis.size), dtype=np.float32)
    for k, v in enumerate(basis.base):
        if v[j] != 0:
            dest = list(v.copy())
            dest[j] -= 1
            dest[i] += 1
            tar = basis.rep_to_index(dest)
            mat[tar, k] = np.sqrt(v[i]+1)*np.sqrt(v[j])
    return mat

@memory.cache    
def bbd(basis, i, j):
    mat = dok_matrix((basis.size, basis.size), dtype=np.float32)
    for k, v in enumerate(basis.base):
        if v[i] != 0:
            dest = list(v.copy())
            dest[i] -= 1
            dest[j] += 1
            tar = basis.rep_to_index(dest)
            mat[tar, k] = np.sqrt(v[j]+1)*np.sqrt(v[i])
    return mat

# Matrices de aniquilación y creación.Toman la base de origen y destino (basis_o, basis_d) resp
@memory.cache   
def b(basis_o, basis_d, i):
    mat = dok_matrix((basis_d.size, basis_o.size), dtype=np.float32)
    for k, v in enumerate(basis_o.base):
        if v[i] != 0:
            dest = list(v.copy())
            dest[i] -= 1   
            tar = basis_d.rep_to_index(dest)
            mat[tar, k] = np.sqrt(v[i])
    return mat

@memory.cache   
def bd(basis_o, basis_d, i):
    mat = dok_matrix((basis_d.size, basis_o.size), dtype=np.float32)
    for k, v in enumerate(basis_o.base):
        dest = list(v.copy())
        dest[i] += 1   
        tar = basis_d.rep_to_index(dest)
        mat[tar, k] = np.sqrt(v[i]+1)
    return mat

# Devuelve la matriz gamma asociada a la descomposición (M,N-M) del vector
def gamma(basis, m, vect):
    d = basis.d
    m_basis = fixed_basis(m, d)
    nm_basis = fixed_basis(basis.m-m,d)
    mat = dok_matrix((m_basis.size, nm_basis.size), dtype=np.float32)
    for i, v in enumerate(m_basis.base):
        for j, w in enumerate(nm_basis.base):
            targ = v+w
            index = basis.rep_to_index(targ)
            if index == None:
                continue
            coef = vect[index]
            if coef != 0:      
                aux = lambda x: np.prod(np.reciprocal(np.sqrt([np.math.factorial(o) for o in x])))
                coef = coef * aux(v) * aux(w)
                #print(v,w,coef)
            mat[i,j] = coef
    return mat

# Devuelve la matriz rho M asociada al vector
def rho_m(basis, m, vect):
    g = gamma(basis, m, vect)
    return np.dot(g,np.transpose(g))

In [ ]:
m = 4
d = 4
a = fixed_basis(m, d)
print(a.base)
a.rep_to_index(np.array([0, 1, 1]))

In [ ]:
# Prueba de las matrices gamma (TODO normalizacion)
v = a.rep_to_vect([0, 1, 1, 2])+a.rep_to_vect([1, 1, 2, 0])
print(v)
gamma(a, 2, v)
v = np.array([random.random() for a in range(0,a.size)])
v = v / np.linalg.norm(v)
print(v)
gamma(a, 2, v)

In [ ]:
# Prueba de funcionamiento de las matrices bdb y bbd
v = [1, 1, 0, 2]
vect = a.rep_to_vect(v)
res = bbd(a,0,1) * vect
if np.linalg.norm(res) != 0:
    res = res / np.linalg.norm(res)
    print(a.vect_to_repr(res))
else:
    res = 0
    print(0)


In [ ]:
# Prueba de funcionamiento de las matrices b y bd

l = fixed_basis(3, 3)
m = fixed_basis(2, 3)

vect = l.rep_to_vect([0,2,1])
res = b(l,m,2) * vect
if np.linalg.norm(res) != 0:
    res = res / np.linalg.norm(res)
    #print(m.vect_to_repr(res))
    
vect = m.rep_to_vect([1,0,1])
res = bd(m,l,2) * vect
if np.linalg.norm(res) != 0:
    res = res / np.linalg.norm(res)
    print(l.vect_to_repr(res))

## Pairing

Construimos el espacio para n particulas en d estados. El hamiltoniano de pairing posee energias equiespaciadas con degeneración doble (asociada a cada estado y su inverso temporal), así como un término de interacción. Por tal motivo, crearemos un espacio con d = 2d.

In [4]:
m = 4
d = 2
# Creo las bases para no tener que recrearlas luego
basis = fixed_basis(m, 2*d)
basis_m1 = fixed_basis(m-1, 2*d)
basis_m2 = fixed_basis(m-2, 2*d)
print(basis.base)



[[0 0 0 4]
 [0 0 1 3]
 [0 0 2 2]
 [0 0 3 1]
 [0 0 4 0]
 [0 1 0 3]
 [0 1 1 2]
 [0 1 2 1]
 [0 1 3 0]
 [0 2 0 2]
 [0 2 1 1]
 [0 2 2 0]
 [0 3 0 1]
 [0 3 1 0]
 [0 4 0 0]
 [1 0 0 3]
 [1 0 1 2]
 [1 0 2 1]
 [1 0 3 0]
 [1 1 0 2]
 [1 1 1 1]
 [1 1 2 0]
 [1 2 0 1]
 [1 2 1 0]
 [1 3 0 0]
 [2 0 0 2]
 [2 0 1 1]
 [2 0 2 0]
 [2 1 0 1]
 [2 1 1 0]
 [2 2 0 0]
 [3 0 0 1]
 [3 0 1 0]
 [3 1 0 0]
 [4 0 0 0]]


In [8]:
# Parametros hamiltoniano
e = 1
eps = 0.01
e0 = np.zeros(2*d)
for k in range(0, d):
    r = random.random() * eps
    e0[2*k] = k*e+r
    e0[2*k+1] = k*e+r

def hamiltonian(g, basis, basis_m1, basis_m2):
    # Construccion de H
    h0 = sum([e0[k]*bdb(basis,k,k) for k in range(0,2*d)])
    hi = dok_matrix((basis.size, basis.size), dtype=np.float32)
    for k in range(0,d):
        for kb in range(0,d):
            hi += g*bd(basis_m1, basis, k)*bd(basis_m2, basis_m1, k+1)*b(basis_m1, basis_m2, kb+1)*b(basis, basis_m1, kb)

    return h0+hi

def solve(g, m):
    h = hamiltonian(g, basis, basis_m1, basis_m2)
    sol = linalg.eigs(h,which='SM',k=20,return_eigenvectors=True)
    argmin = np.argmin(sol[0].real)
    fund = sol[1][:,argmin]
    fund = fund.real / np.linalg.norm(fund)
    rho = rho_m(basis, m, fund).todense()
    return linalg_d.eigvals(rho).real

In [18]:
m = 2
num = 200
g_range = np.linspace(0.01,1,num)

size = int(binom(2*d+m-1,2*d-1))
rho_range = np.zeros((num,size))
for i, g in enumerate(g_range):
    print(g)
    rho_range[i,:] = solve(g,m)

0.01
0.014974874371859296
0.019949748743718593
0.024924623115577887
0.029899497487437185
0.03487437185929648
0.03984924623115578
0.04482412060301508
0.049798994974874375
0.05477386934673367
0.05974874371859297
0.06472361809045227
0.06969849246231155
0.07467336683417085
0.07964824120603015
0.08462311557788944
0.08959798994974874
0.09457286432160804
0.09954773869346734
0.10452261306532663
0.10949748743718593
0.11447236180904523
0.11944723618090453
0.12442211055276382
0.12939698492462312
0.1343718592964824
0.13934673366834172
0.14432160804020103
0.1492964824120603
0.15427135678391962
0.1592462311557789
0.16422110552763822
0.1691959798994975
0.17417085427135678
0.1791457286432161
0.18412060301507538
0.1890954773869347
0.19407035175879397
0.19904522613065329
0.20402010050251257
0.20899497487437188
0.21396984924623116
0.21894472361809048
0.22391959798994976
0.22889447236180907
0.23386934673366835
0.23884422110552767
0.24381909547738695
0.24879396984924623
0.2537688442211055
0.258743718592964

In [19]:
# Ploteamos
fig = go.Figure()
for x in range(0,size):
    fig.add_trace(go.Scatter(
        x=g_range,
        y=rho_range[:,x]
    ))
fig.update_layout(xaxis_title='G',
                   yaxis_title='Rho2')
fig.show()

In [ ]:
rho_range

In [ ]:
int(binom(4+2-1,4-1))